<style>
    .title {
        font-weight: 'bold';
    }
</style>

# <span class="title">08.1 - Création du dataset de données traitées pour entrainement </br> avec criticités équilibrées</span>
(sans encodage, ni normalisation)

Ce notebook génère 1 csv :

- dataset_for_training.csv : analyse et nettoyage des variables explicatives

Etapes : 




## A) Imports

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# source path to datasets
path = '../data/'
data = 'dataset_for_preprocess_criticality_balanced_07.1.csv'
save_csv = '../data/dataset_for_training_criticality_balanced_08.1.csv'

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(os.path.join(path, data), header=0, parse_dates=True, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23356 entries, 0 to 35997
Data columns (total 60 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   total_copies_job                                       23356 non-null  int64  
 1   started_at                                             23356 non-null  object 
 2   ended_at                                               23356 non-null  object 
 3   speed_job                                              23356 non-null  int64  
 4   paperHeight_job                                        23356 non-null  int64  
 5   paperWidth_job                                         23356 non-null  int64  
 6   total_copies_requested_job                             23356 non-null  int64  
 7   ifoil_job                                              23356 non-null  bool   
 8   scanner_mode                                  

## B) Analyse des variables

In [4]:
# remise à zero des index
df.reset_index(drop=True,inplace=True)

In [5]:
# suppression des colonnes de metrics
# metrics_cols = [
#     'status', 
#     'source_events', 
#     'timestamp', 
#     'criticality_events', 
#     'name_modules', 
#     'type_modules',
#     'generation_modules', 
#     'value_counters_modules', 
#     'name_counters_modules',
#     'name_connected_operators',
#     'level_connected_operators',
#     'varnishLevelsTargetvolume',
#     'varnishLevelsTotalvolume'
#     ]
metrics_cols = [
    'timestamp', 
    'identification_events', 
    'index',
    ]
df.drop(columns=metrics_cols, axis=0, inplace=True)

In [6]:
# suppression des colonnes en doublon entre job_events et jobs
job_events_cols = [
    'bars_event',
    'paperWidth_event', 
    'paperHeight_event',
    'varnishConsumptionVarnish_3d_event'
    ]
df.drop(columns=job_events_cols, axis=0, inplace=True)

In [7]:
# suppression des colonnes de variables catégorielles
cat_cols = [
    'operator_level', 
    'operator',
    'paperName', 
    'jobState',
    'jobId'
   ]
df.drop(columns=cat_cols, axis=0, inplace=True)

In [8]:
# suppression des colonnes datetime et ajout d'une colonne duration
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
# Calculer la différence de temps entre les deux colonnes
df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df.drop(columns=['started_at', 'ended_at'], axis=0, inplace=True)

In [9]:
df.head(3)

,total_copies_job,speed_job,paperHeight_job,paperWidth_job,total_copies_requested_job,ifoil_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,copies_per_run,total_copies_requested_event,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil_event,optifoil_ifoil,vacuumIn_ifoil,vacuumOut_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,speedTensionIn_ifoil,speedTensionOut_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed_event,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,x_cropmark1_cropmarksMode_remoteScannerRegistration,y_cropmark1_cropmarksMode_remoteScannerRegistration,x_cropmark2_cropmarksMode_remoteScannerRegistration,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies_event,criticality_events,duration
0,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,False,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,False,3,5,NO_EVENT,630.565
1,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,False,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,False,3,5,NO_EVENT,630.565
2,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,False,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,False,3,5,NO_EVENT,630.565


In [10]:
# Liste des colonnes à vérifier
colonnes_a_verifier = ['dithering','deadPixelsOffset','speedTensionIn_ifoil']

# Vérifier le nombre de valeurs uniques pour chaque colonne dans la liste
nombres_uniques = df[colonnes_a_verifier].nunique()

print(nombres_uniques)

dithering                2
deadPixelsOffset         2
speedTensionIn_ifoil    10
dtype: int64


## C) Nettoyage

In [11]:
# supression des colonnes dont les valeurs sont constantes
# contstant_cols = ['dithering','speedTensionIn_ifoil']

constant_cols = [col for col in df.columns if df[col].nunique() == 1]
print(constant_cols)
df.drop(columns=constant_cols, axis=0, inplace=True)

['optifoil_ifoil', 'enable_specialSubstrate_remoteScannerRegistration']


In [12]:
df.head(5)

,total_copies_job,speed_job,paperHeight_job,paperWidth_job,total_copies_requested_job,ifoil_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,copies_per_run,total_copies_requested_event,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil_event,vacuumIn_ifoil,vacuumOut_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,speedTensionIn_ifoil,speedTensionOut_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed_event,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,x_cropmark1_cropmarksMode_remoteScannerRegistration,y_cropmark1_cropmarksMode_remoteScannerRegistration,x_cropmark2_cropmarksMode_remoteScannerRegistration,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies_event,criticality_events,duration
0,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,3,5,NO_EVENT,630.565
1,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,3,5,NO_EVENT,630.565
2,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,3,5,NO_EVENT,630.565
3,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,3,5,NO_EVENT,630.565
4,5,313,483,330,5,True,3,2,0.321059,0,0,0,5,40,4,False,1,20.0,True,100,100,"[{'id': 1, 'end': 483, 'start': 0, 'height': 4...",True,-0.6,1.0,95,1487,-69,313,20,60,1500,1500,0,0,0,0,100,1500,24,25,3,5,ERROR,630.565


In [13]:
# Vérifier si toutes les valeurs dans les deux colonnes sont identiques
same_col = (df['total_copies_job'] == df['total_copies_event']).all()
print("Les colonnes sont-elles identiques ?", same_col)


Les colonnes sont-elles identiques ? True


In [14]:
df.shape

(23356, 45)

In [15]:
# Supprimer la colonne 'total_copies_event'
df.drop(columns='total_copies_event', inplace=True)

df.rename(columns={'total_copies_job': 'total_copies', 'ifoil_job': 'ifoil' , 'speed_job': 'speed','total_copies_requested_job' : 'total_copies_requested'}, inplace=True)


In [16]:
df.shape

(23356, 44)

<style>
    .important {color: #A5EF6E;}
    .purple { color: #EF6EE9;}
</style>

Données disponibles au démarrage d'un job :
- <span class='important'>1  thumbnail* : chemin vignette projet</span>
- 2  total_copies : nombre de copies imprimées
- 3  started_at : date de début
- 4  ended_at : date de fin
- <span class='important'>5  machineId* : identifiant machine</span>
- 6  speed : vitesse d'impression
- 7  operator : opérateur
- 8  operator_level : niveau de l'opérateur
- 9  first_page_image_path_on_machine* : chemin de l'image
- 10 paperHeight : hauteur du substrat
- 11 paperWidth : largeur du substrat
- 12 paperName : appelation du substrat
- <span class='important'>13 paperThickness* : épaisseur du substrat (valeur unique à 0)</span>
- 14 id_on_machine : identifiant unique du travail d'impression
- 15 total_copies_requested : nombre de copies demandées
- <span class='important'>16 job_thumbnail_id* : identifiant de vignette image</span>
- 17 uses_ifoil : impression utilisant de la dorure
- <span class='important'>18 uses_iper* : impression nécéssitant l'iper (valeur unique à True)</span>
- 19 scanner_mode : niveau de config du scanner
- 20 iper_bvar_count : compteur <span class='purple'># TODO vérifier la fusion notebook 06</span>
- 21 varnishConsumptionVarnish_3d : consommation de vernis en 3d
- <span class='important'>22 varnishConsumptionVarnish_2d* : consommation de vernis en 2d</span>

<span class='important'> *\*variables déjà supprimées dans le notebook 05 ou 06*</span>

In [17]:
# colonnes à conserver dispos dans jobs
jobs_cols_to_keep = [
    'duration',
    'total_copies',
    'speed',
    'paperHeight_job', 
    'paperWidth_job', 
    'total_copies_requested',
    'ifoil',
    'scanner_mode', 
    'bars_job',
    'varnishConsumptionVarnish_3d_job'
    ]
# colonnes à conserver dispos dans job events avec le tag start
jobevents_cols_to_keep = [
    'LED',
    'drops', 
    'speed_ifoil',
    'stampAreas_ifoil',
    'heater1Enabled_ifoil', 
    'heater1Temperature_ifoil',
    'x_imageLayout',
    'y_imageLayout', 
    'power_irDryers', 
    'power_uvDryers',
    'redScore_gridMode_remoteScannerRegistration',
    'redScore_cropmarksMode_remoteScannerRegistration',
    'redScore_fullScannerMode_remoteScannerRegistration',
    'blueScore_fullScannerMode_remoteScannerRegistration',
    'greenScore_fullScannerMode_remoteScannerRegistration',
    'mode_remoteScannerRegistration'
    ]
# colonnes à conserver pour la prédiction
metrics_cols_to_keep = ['criticality_events']

# fusion des listes de colonnes à conserver
cols_to_keep = jobs_cols_to_keep + jobevents_cols_to_keep + metrics_cols_to_keep

# dataframe des variables conservées
df_to_encode = df[cols_to_keep].copy()

In [18]:
df_to_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23356 entries, 0 to 23355
Data columns (total 27 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   duration                                              23356 non-null  float64
 1   total_copies                                          23356 non-null  int64  
 2   speed                                                 23356 non-null  int64  
 3   paperHeight_job                                       23356 non-null  int64  
 4   paperWidth_job                                        23356 non-null  int64  
 5   total_copies_requested                                23356 non-null  int64  
 6   ifoil                                                 23356 non-null  bool   
 7   scanner_mode                                          23356 non-null  int64  
 8   bars_job                                              23

In [19]:
df_to_encode.drop(columns=['stampAreas_ifoil', 'total_copies', 'mode_remoteScannerRegistration'], axis=0, inplace=True)

In [20]:
df_to_encode.criticality_events.unique()

array(['NO_EVENT', 'ERROR', 'INFO', 'WARNING'], dtype=object)

## D) Output

In [21]:
df_to_encode.to_csv(save_csv)